# This is a demonstrator of the Score-P Python Kernel
This is the Score-P Python Kernel that allows you to execute Jupyter Notebooks with Score-P.

The kernel supports the usual jupyter interactivity between cells but with some limitations (see "General Limitations").

## Setup
You can set up your Score-P environment by executing a cell that starts with the %%scorep_env magic command.

You can set the Score-P Python binding arguments by executing a cell that starts with %%scorep_python_binding_arguments.

## Usage
Cells that should be executed with Score-P have to be marked with %%execute_with_scorep in the first line. Cells without that command are executed as ordinary Python processes.

### Multi Cell Mode
You can also treat multiple cells as one single cell by using the multi cell mode.

Therefore you can mark the cells in the order you wish to execute them. Start the marking process by a cell that starts with the %%enable_multicellmode command.
Now mark your cells by running them. Note that the cells will not be executed at this point but will be marked for later execution.
You can stop the marking and execute all the marked cells by running a cell that starts with %%finalize_multicellmode command.
This will execute all the marked cells orderly with Score-P. Note that the %%execute_with_scorep command has no effect in the multi cell mode.

There is no "unmark" command available but you can abort the multicellmode by the %%abort_multicellmode command. Start your marking process again if you have marked your cells in the wrong order.

The %%enable_multicellmode, %%finalize_multicellmode and %%abort_multicellmode commands should be run in an exclusive cell. Additional code in the cell will be ignored.

### Presentation of Performance Data

To inspect the collected performance data, use tools as Vampir (Trace) or Cube (Profile).

---
# Set up SCORE-P Environment

In [13]:
%%scorep_env
SCOREP_ENABLE_TRACING=1
SCOREP_ENABLE_PROFILING=0
SCOREP_TOTAL_MEMORY=3g

Score-P environment set successfully: {'SCOREP_ENABLE_TRACING': '1', 'SCOREP_ENABLE_PROFILING': '0', 'SCOREP_TOTAL_MEMORY': '3g'}

In [14]:
%%scorep_python_binding_arguments
--noinstrumenter

Score-P Python binding arguments set successfully: ['--noinstrumenter', '--noinstrumenter']

In [15]:
import scorep
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import sys

---
# Example 1: Data Conjunction

In [16]:
nrows, ncols = 1000000, 20
df1, df2, df3, df4 = [pd.DataFrame(np.random.randn(nrows, ncols)) for _ in range(4)]

In [17]:
%%execute_with_scorep
with scorep.instrumenter.enable():
    # data conjunction
    df5 = df1 + df2 +df3 + df4

 Instrumentation results can be found in /home/visitor/Demonstrators/score-p_kernel/supplementary/example/scorep-20230630_1737_31341656626898

In [ ]:
%%execute_with_scorep
with scorep.instrumenter.enable():
    # data conjunction
    df5 = pd.eval("df1 + df2 +df3 + df4", engine='numexpr')

 Instrumentation results can be found in /home/visitor/Demonstrators/score-p_kernel/supplementary/example/scorep-20230630_1737_31353813505508

---
# Example 2: Deep Learning

In [ ]:
filename = "fairytales_demo.txt"

In [ ]:
# imports
import scorep
import logging

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%d/%m/%Y %H:%M:%S",
    level=logging.INFO)

from utils import set_seed
set_seed(42)

import numpy as numpy
import torch
import torch.nn as nn
from torch.nn import functional as F

import math
from torch.utils.data import Dataset

from model import GPT, GPTconfig
from trainer import Trainer, TrainerConfig

In [ ]:
# defining the data set
class CharDataset(Dataset):
    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print("data has %d characters, %d unique." % (data_size, vocab_size))

        self.stoi = {ch:i for i, ch in enumerate(chars)}
        self.itos = {i:ch for i, ch in enumerate(chars)}
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data

    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        chunk = self.data[idx : idx+self.block_size+1]
        dix = [self.stoi[s] for s in chunk]

        x = torch.tensor(dix[:-1], dtype = torch.long)
        y = torch.tensor(dix[1:], dtype = torch.long)
        return x, y

In [ ]:
%%execute_with_scorep

with scorep.instrumenter.enable():
    block_size = 32

    text = open("./{}".format(filename), "r").read()
    train_dataset = CharDataset(text, block_size)

    
    mconf = GPTconfig(train_dataset.vocab_size, train_dataset.block_size,
                      n_layer=4, n_head=4, n_embd=256)
    model = GPT(mconf)

    tconf = TrainerConfig(max_epochs=1, batch_size=1024, learning_rate=0.01,
                          lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*block_size,
                          num_workers=1)
    trainer = Trainer(model, train_dataset, None, tconf)

    torch.cuda.empty_cache()
    trainer.train()

    torch.save(model.state_dict(), "./saved_models/trained_gpt_model")

![image info](trace_profile.png)

## Try your model

In [ ]:
context = "The sun shone in the sky."

In [ ]:
%%execute_with_scorep
from utils import sample

with scorep.instrumenter.enable():
    x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
    y = sample(model, x, 2000, temperature=1.0, sample=True, top_k=10)[0]

    completion = ''.join([train_dataset.itos[int(i)] for i in y])
    print(completion)

---
# Playground
Feel free to add your code and to analyze it

Begin a cell with:
%%execute_with_scorep